# 5G NR PUSCH Tutorial

This notebook provides an introduction to Sionna's [5G New Radio (NR) module](https://nvlabs.github.io/sionna/api/nr.html) and, in particular, the [physical uplink shared channel (PUSCH)](https://nvlabs.github.io/sionna/api/nr.html#pusch). This module provides implementations of a small subset of the physical layer functionalities as described in the 3GPP specifications [38.211](https://portal.3gpp.org/desktopmodules/Specifications/SpecificationDetails.aspx?specificationId=3213), [38.212](https://portal.3gpp.org/desktopmodules/Specifications/SpecificationDetails.aspx?specificationId=3214) and [38.214](https://portal.3gpp.org/desktopmodules/Specifications/SpecificationDetails.aspx?specificationId=3216). 


You will

- Get an understanding of the different components of a PUSCH configuration, such as the carrier, DMRS, and transport block,
- Learn how to rapidly simulate PUSCH transmissions for multiple transmitters,
- Modify the PUSCHReceiver to use a custom MIMO Detector.

## Table of Contents
* [GPU Configuration and Imports](#GPU-Configuration-and-Imports)
* [A "Hello World!" Example](#A-Hello-World-Example)
* [Carrier Configuration](#Carrier-Configuration)
* [Understanding the DMRS Configuration](#Understanding-the-DMRS-Configuration)
    * [Configuring Multiple Layers](#Configuring-Multiple-Layers)
    * [Controlling the Number of DMRS Symbols in a Slot](#Controlling-the-Number-of-DMRS-Symbols-in-a-Slot)
    * [How to control the number of available DMRS ports?](#How-to-control-the-number-of-available-DMRS-ports?)
* [Transport Blocks and MCS](#Transport-Blocks-and-MCS)
* [Looking into the PUSCHTransmitter](#Looking-into-the-PUSCHTransmitter)
* [Components of the PUSCHReceiver](#Components-of-the-PUSCHReceiver)
* [End-to-end PUSCH Simulations](#End-to-end-PUSCH-Simulations)


## GPU Configuration and Imports

In [41]:
import os
if os.getenv("CUDA_VISIBLE_DEVICES") is None:
    gpu_num = 0 # Use "" to use the CPU
    os.environ["CUDA_VISIBLE_DEVICES"] = f"{gpu_num}"
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
# tahtan
# Import Sionna
import sys
sys.path.append('../')
import sionna

# try:
#     import sionna
# except ImportError as e:
#     # Install Sionna if package is not already installed
#     import os
#     os.system("pip install sionna")
#     import sionna

import tensorflow as tf
# Configure the notebook to use only a single GPU and allocate only as much memory as needed
# For more details, see https://www.tensorflow.org/guide/gpu
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e:
        print(e)
# Avoid warnings from TensorFlow
tf.get_logger().setLevel('ERROR')

sionna.config.seed = 42 # Set seed for reproducible results

# Load the required Sionna components
from sionna.nr import PUSCHConfig, PUSCHTransmitter, PUSCHReceiver, CarrierConfig, PUSCHDMRSConfig,\
                        TBConfig, PUSCHPilotPattern, TBEncoder, PUSCHPrecoder, LayerMapper, LayerDemapper, check_pusch_configs,\
                        TBDecoder, PUSCHLSChannelEstimator
from sionna.nr.utils import generate_prng_seq
from sionna.channel import AWGN, RayleighBlockFading, OFDMChannel, TimeChannel, time_lag_discrete_time_channel
from sionna.channel.utils import * 
from sionna.channel.tr38901 import Antenna, AntennaArray, UMi, UMa, RMa, TDL, CDL
from sionna.channel import gen_single_sector_topology as gen_topology
from sionna.utils import compute_ber, ebnodb2no, sim_ber, array_to_hash, create_timestamped_folders, b2b, f2f, \
    BinarySource, config_parser, fft_size_return
from sionna.ofdm import KBestDetector, LinearDetector, MaximumLikelihoodDetector,\
        LSChannelEstimator, LMMSEEqualizer, RemoveNulledSubcarriers, ResourceGridDemapper,\
        ResourceGrid, ResourceGridMapper, OFDMModulator
from sionna.mimo import StreamManagement
from sionna.mapping import Mapper, Demapper


In [17]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time
from datetime import datetime, timedelta
# from bs4 import BeautifulSoup
import pickle
from collections import namedtuple
import json
from tqdm.notebook import tqdm
import itertools
import io

## A Hello World Example

Let us start with a simple "Hello, World!" example in which we will simulate PUSCH transmissions from a single transmitter to a single receiver over an AWGN channel.

In [36]:
_num_tx = 1
_num_rx = 1
_num_tx_ant = 1
_num_rx_ant = 8
_carrier_frequency = 2.55e9  # Carrier frequency in Hz.
_link_direction = "uplink"

# Configure antenna arrays
_ue_antenna = Antenna(polarization="single",
                polarization_type="V",
                antenna_pattern="38.901",
                carrier_frequency=_carrier_frequency)

_gnb_array = AntennaArray(num_rows=1,
                        num_cols=_num_rx_ant//2,
                        polarization="dual",
                        polarization_type="cross",
                        antenna_pattern="38.901",
                        carrier_frequency=_carrier_frequency)

_channel_model_0 = CDL("C", 150e-9, _carrier_frequency, _ue_antenna, _gnb_array, _link_direction, min_speed=10)

In [8]:
cfg_path = f'../Pusch_data/case/case_{20002}.cfg'
caseInfo, sysInfo, ue, chcfg, auxInfo = config_parser(cfg_path)
caseInfo, sysInfo, ue, chcfg, auxInfo

({'sfn': 6, 'sf': 4},
 {'NCellId': 246,
  'FrequencyRange': 1,
  'BandWidth': 100,
  'Numerology': 1,
  'CpType': 0,
  'NTxAnt': 1,
  'NRxAnt': 8,
  'BwpNRb': 273,
  'BwpRbOffset': 0,
  'harqProcFlag': 0,
  'nHarqProc': 1,
  'rvSeq': 0},
 {1: {'rvIdx': 0,
   'TransformPrecoding': 0,
   'Rnti': 20002,
   'nId': 246,
   'CodeBookBased': 0,
   'DmrsPortSetIdx': 1,
   'NLayers': 1,
   'NumDmrsCdmGroupsWithoutData': 2,
   'Tpmi': 0,
   'FirstSymb': 0,
   'NPuschSymbAll': 14,
   'RaType': 1,
   'FirstPrb': 0,
   'NPrb': 4,
   'FrequencyHoppingMode': 0,
   'McsTable': 0,
   'Mcs': 3,
   'ILbrm': 0,
   'nScId': 0,
   'NnScIdId': 246,
   'DmrsConfigurationType': 0,
   'DmrsDuration': 1,
   'DmrsAdditionalPosition': 1,
   'PuschMappingType': 0,
   'DmrsTypeAPosition': 3,
   'HoppingMode': 0,
   'NRsId': 0,
   'Ptrs': 0,
   'ScalingFactor': 0,
   'OAck': 0,
   'IHarqAckOffset': 11,
   'OCsi1': 0,
   'ICsi1Offset': 7,
   'OCsi2': 0,
   'ICsi2Offset': 0,
   'NPrbOh': 0,
   'nCw': 1,
   'TpPi2Bpsk':

In [ ]:
# class MyPUSCHConfig(PUSCHConfig):
#     def __init__(self, sysInfo, ue):
#         super().__init__(
#             carrier_config=CarrierConfig(
#                 n_cell_id=sysInfo['NCellId'],
#                 cyclic_prefix="normal",
#                 subcarrier_spacing=30,
#                 n_size_grid=sysInfo['BwpNRb'],
#                 n_start_grid=sysInfo['BwpRbOffset'],
#                 slot_number=0,
#                 frame_number=0
#             ),
#             pusch_dmrs_config=PUSCHDMRSConfig(
#                 config_type=ue[1]['DmrsConfigurationType'] + 1,
#                 length=ue[1]['DmrsDuration'],
#                 additional_position=ue[1]['DmrsAdditionalPosition'],
#                 dmrs_port_set=[1],
#                 n_id=ue[1]['NnScIdId'],
#                 n_scid=ue[1]['nScId'],
#                 num_cdm_groups_without_data=2,
#                 type_a_position=ue[1]['DmrsTypeAPosition']
#             ),
#             tb_config=TBConfig(
#                 channel_type='PUSCH',
#                 n_id=ue[1]['nId'],
#                 mcs_table=ue[1]['McsTable'] + 1,
#                 mcs_index=ue[1]['Mcs']
#             ),
#             mapping_type='A',
#             n_size_bwp=ue[1]['NPrb'],
#             n_start_bwp=ue[1]['FirstPrb'],
#             num_layers=1,
#             num_antenna_ports=1,
#             precoding='non-codebook',
#             tpmi=ue[1]['Tpmi'],
#             transform_precoding=False,
#             n_rnti=ue[1]['Rnti'],
#             symbol_allocation=[0,14]
#         )

# _pusch_config_0 = MyPUSCHConfig(sysInfo, ue)
# _pusch_config_0.show()

Carrier Configuration
cyclic_prefix : normal
cyclic_prefix_length : 2.8645833333333334e-06
frame_duration : 0.01
frame_number : 0
kappa : 64.0
mu : 1
n_cell_id : 246
n_size_grid : 273
n_start_grid : 0
num_slots_per_frame : 20
num_slots_per_subframe : 2
num_symbols_per_slot : 14
slot_number : 0
sub_frame_duration : 0.001
subcarrier_spacing : 30
t_c : 5.086263020833334e-10
t_s : 3.2552083333333335e-08

PUSCH Configuration
dmrs_grid : shape (1, 48, 14)
dmrs_grid_precoded : shape ()
dmrs_mask : shape (48, 14)
dmrs_symbol_indices : [3, 11]
frequency_hopping : neither
l : [3, 11]
l_0 : 3
l_bar : [3, 11]
l_d : 14
l_prime : [0]
l_ref : 0
mapping_type : A
n : shape (12,)
n_rnti : 20002
n_size_bwp : 4
n_start_bwp : 0
num_antenna_ports : 1
num_coded_bits : 1152
num_layers : 1
num_ov : 0
num_res_per_prb : 144
num_resource_blocks : 4
num_subcarriers : 48
precoding : non-codebook
precoding_matrix : None
symbol_allocation : [0, 14]
tb_size : 288
tpmi : 0
transform_precoding : False

PUSCH DMRS Config

In [22]:
class MyPUSCHConfig(PUSCHConfig):
    def __init__(self):
        super().__init__(
            carrier_config=CarrierConfig(
                n_cell_id=0,
                cyclic_prefix="normal",
                subcarrier_spacing=30,
                n_size_grid=273,
                n_start_grid=0,
                slot_number=4,
                frame_number=0
            ),
            pusch_dmrs_config=PUSCHDMRSConfig(
                config_type=1,
                length=1,
                additional_position=1,
                dmrs_port_set=[0],
                n_id=0,
                n_scid=0,
                num_cdm_groups_without_data=2,
                type_a_position=2
            ),
            tb_config=TBConfig(
                channel_type='PUSCH',
                n_id=0,
                mcs_table=1,
                mcs_index=3
            ),
            mapping_type='A',
            n_size_bwp=4,
            n_start_bwp=0,
            num_layers=1,
            num_antenna_ports=1,
            precoding='non-codebook',
            tpmi=0,
            transform_precoding=False,
            n_rnti=2008,
            symbol_allocation=[0,14]
        )

_pusch_config_0 = MyPUSCHConfig()
_pusch_config_0.show()

_tb_size = _pusch_config_0.tb_size
_num_coded_bits = _pusch_config_0.num_coded_bits
_target_coderate = _pusch_config_0.tb.target_coderate
_num_bits_per_symbol = _pusch_config_0.tb.num_bits_per_symbol
_num_layers = _pusch_config_0.num_layers
_n_rnti = _pusch_config_0.n_rnti
_n_id = _pusch_config_0.tb.n_id

_dmrs_length = _pusch_config_0.dmrs.length
_dmrs_additional_position = _pusch_config_0.dmrs.additional_position
_num_cdm_groups_without_data = _pusch_config_0.dmrs.num_cdm_groups_without_data
_n_scid = _pusch_config_0.dmrs.n_scid
_n_id_n_scid = _pusch_config_0.dmrs.n_id[0]

_n_size_bwp = _pusch_config_0.n_size_bwp
_n_start_bwp = _pusch_config_0.n_start_bwp

Carrier Configuration
cyclic_prefix : normal
cyclic_prefix_length : 2.3437500000000002e-06
frame_duration : 0.01
frame_number : 0
kappa : 64.0
mu : 1
n_cell_id : 0
n_size_grid : 273
n_start_grid : 0
num_slots_per_frame : 20
num_slots_per_subframe : 2
num_symbols_per_slot : 14
slot_number : 4
sub_frame_duration : 0.001
subcarrier_spacing : 30
t_c : 5.086263020833334e-10
t_s : 3.2552083333333335e-08

PUSCH Configuration
dmrs_grid : shape (1, 48, 14)
dmrs_grid_precoded : shape ()
dmrs_mask : shape (48, 14)
dmrs_symbol_indices : [2, 11]
frequency_hopping : neither
l : [2, 11]
l_0 : 2
l_bar : [2, 11]
l_d : 14
l_prime : [0]
l_ref : 0
mapping_type : A
n : shape (12,)
n_rnti : 2008
n_size_bwp : 4
n_start_bwp : 0
num_antenna_ports : 1
num_coded_bits : 1152
num_layers : 1
num_ov : 0
num_res_per_prb : 144
num_resource_blocks : 4
num_subcarriers : 48
precoding : non-codebook
precoding_matrix : None
symbol_allocation : [0, 14]
tb_size : 288
tpmi : 0
transform_precoding : False

PUSCH DMRS Configura

In [23]:
"""
frame_number, n_cell_id: No change
n_rnti, tb.n_id: Change in tb_encoder, but not in dmrs (same c -> same x)
slot_number, dmrs.n_id, dmrs.n_scid, mcs: No change in tb_encoder (same b -> same c), but in dmrs
"""

'\nframe_number, n_cell_id: No change\nn_rnti, tb.n_id: Change in tb_encoder, but not in dmrs (same c -> same x)\nslot_number, dmrs.n_id, dmrs.n_scid, mcs: No change in tb_encoder (same b -> same c), but in dmrs\n'

In [24]:
mcs_params = {
    'tb_size': [],
    'num_coded_bits': [],
    'target_coderate': [],
    'num_bits_per_symbol': []
}
for mcs in range(0,10):
    pusch_config_i = _pusch_config_0.clone()
    pusch_config_i.tb.mcs_index = mcs
    mcs_params['tb_size'].append(pusch_config_i.tb_size)
    mcs_params['num_coded_bits'].append(pusch_config_i.num_coded_bits)
    mcs_params['target_coderate'].append(pusch_config_i.tb.target_coderate)
    mcs_params['num_bits_per_symbol'].append(pusch_config_i.tb.num_bits_per_symbol)
mcs_params

{'tb_size': [128, 176, 224, 288, 352, 432, 504, 608, 672, 768],
 'num_coded_bits': [1152,
  1152,
  1152,
  1152,
  1152,
  1152,
  1152,
  1152,
  1152,
  1152],
 'target_coderate': [0.1171875,
  0.1533203125,
  0.1884765625,
  0.2451171875,
  0.30078125,
  0.3701171875,
  0.4384765625,
  0.513671875,
  0.587890625,
  0.6630859375],
 'num_bits_per_symbol': [2, 2, 2, 2, 2, 2, 2, 2, 2, 2]}

In [25]:
slot_params = {
    'pilots': []
}
pusch_slots_in_frame = [4,5,14,15]
num_pusch_slots_in_frame = len(pusch_slots_in_frame)
for slot_index in pusch_slots_in_frame:
    pusch_config_i = _pusch_config_0.clone()
    pusch_config_i.carrier.slot_number = slot_index
    pilot_pattern_i = PUSCHPilotPattern([pusch_config_i], dtype=tf.complex64)
    slot_params['pilots'].append(pilot_pattern_i.pilots)

In [26]:
PuschRecord = namedtuple("PuschRecord", [
    "nSFN", "nSlot", "nPDU", "nGroup", "nUlsch", "nUlcch", "nRachPresent",
    "nRNTI", "nUEId", "nBWPSize", "nBWPStart", "nSubcSpacing", "nCpType", "nULType",
    "nMcsTable", "nMCS", "nTransPrecode", "nTransmissionScheme", "nNrOfLayers",
    "nPortIndex", "nNid", "nSCID", "nNIDnSCID", "nNrOfAntennaPorts",
    "nVRBtoPRB", "nPMI", "nStartSymbolIndex", "nNrOfSymbols", "nResourceAllocType",
    "nRBStart", "nRBSize", "nTBSize", "nRV", "nHARQID", "nNDI", "nMappingType",
    "nDMRSConfigType", "nNrOfCDMs", "nNrOfDMRSSymbols", "nDMRSAddPos",
    "nPTRSPresent", "nAck", "nAlphaScaling", "nBetaOffsetACKIndex", "nCsiPart1",
    "nBetaOffsetCsiPart1Index", "nCsiPart2", "nBetaOffsetCsiPart2Index",
    "nTpPi2BPSK", "nTPPuschID", "nRxRUIdx", "nUE", "nPduIdx",

    # New fields for channel and filenames
    "Channel_model", "Speed", "Delay_spread", "Esno_db",
    "Data_filename","Data_dirname"
    ]
)

def save_pickle(data, filename):
    """Saves data to a pickle file."""
    with open(filename, "wb") as f:
        pickle.dump(data, f)

In [27]:
# _binary_source = BinarySource(dtype=tf.float32)

# _mapper = Mapper("qam", _num_bits_per_symbol, dtype=tf.complex64)

# _layer_mapper = LayerMapper(num_layers=_num_layers, dtype=tf.complex64)

# _pilot_pattern = PUSCHPilotPattern([_pusch_config_0],
#                                         dtype=tf.complex64)
# _mu = 1
# _num_ofdm_symbols = 14
# _fft_size = 4096
# _cyclic_prefix_length = 288
# _subcarrier_spacing = 30e3
# _num_guard_subcarriers = (410, 410)
# _num_slots_per_frame = 20

# # Define the resource grid.
# _resource_grid = ResourceGrid(
#     num_ofdm_symbols=_num_ofdm_symbols,
#     fft_size=_fft_size,
#     subcarrier_spacing=_subcarrier_spacing,
#     num_tx=1,
#     num_streams_per_tx=1,
#     cyclic_prefix_length=_cyclic_prefix_length,
#     num_guard_carriers=_num_guard_subcarriers,
#     dc_null=False,
#     pilot_pattern=_pilot_pattern,
#     dtype=tf.complex64
# )
# _resource_grid_mapper = ResourceGridMapper(_resource_grid, dtype=tf.complex64)

# _channel_estimator = PUSCHLSChannelEstimator(
#                 _resource_grid,
#                 _dmrs_length,
#                 _dmrs_additional_position,
#                 _num_cdm_groups_without_data,
#                 interpolation_type='lin',
#                 dtype=tf.complex64)

# _rx_tx_association = np.ones([_num_rx, _num_tx], bool)
# _stream_management = StreamManagement(_rx_tx_association, _num_layers)
# _mimo_detector = LinearDetector("lmmse", "bit", "maxlog", _resource_grid,
#                                 _stream_management, "qam", _num_bits_per_symbol, dtype=tf.complex64)

# def generate_data(name,
#         data_dir,
#         mcss,
#         cdl_models,
#         speeds,
#         delay_spreads,
#         esno_dbs,
#         samples_per_case=1):
    
#     pusch_records=[]
#     parquet_dir = f'{data_dir}/parquet'

#     pickle_dir = f'{data_dir}/pickle/{name}'
#     os.makedirs(pickle_dir, exist_ok=True)

#     total_iterations = len(mcss) * len(cdl_models) * len(speeds) * len(delay_spreads) * len(esno_dbs) * samples_per_case
#     with tqdm(total=total_iterations, desc="Generating Data") as pbar:
#         for mcs in mcss:
#             tb_size = mcs_params['tb_size'][mcs]
#             num_coded_bits = mcs_params['num_coded_bits'][mcs]
#             target_coderate = mcs_params['target_coderate'][mcs]
#             num_bits_per_symbol = mcs_params['num_bits_per_symbol'][mcs]
#             tb_encoder = TBEncoder(
#                         target_tb_size=tb_size,
#                         num_coded_bits=num_coded_bits,
#                         target_coderate=target_coderate,
#                         num_bits_per_symbol=num_bits_per_symbol,
#                         num_layers=_num_layers,
#                         n_rnti=_n_rnti,
#                         n_id=_n_id,
#                         channel_type="PUSCH",
#                         codeword_index=0,
#                         use_scrambler=True,
#                         verbose=False,
#                         output_dtype=tf.float32)
            
#             for cdl_model, speed, delay_spread in itertools.product(cdl_models, speeds, delay_spreads):
#                 channel_model_i = CDL(cdl_model, delay_spread, _carrier_frequency, _ue_antenna, _gnb_array, _link_direction, min_speed=speed)
#                 channel_i = OFDMChannel(channel_model_i, _resource_grid, return_channel=True)
                
#                 for esno_db in esno_dbs:
#                     No = pow(10., -esno_db / 10.)
                    
#                     for sample_num in range(samples_per_case):
#                         status_str = f"(MCS {mcs} | CDL-{cdl_model} | {speed} m/s | {delay_spread} s) | {esno_db} dB | Sample {sample_num+1}/{samples_per_case}"
#                         pbar.set_description(status_str)
                        
#                         slot_num = pusch_slots_in_frame[sample_num % num_pusch_slots_in_frame]
#                         _resource_grid_mapper._resource_grid.pilot_pattern.pilots = slot_params['pilots'][sample_num % num_pusch_slots_in_frame]

#                         b = _binary_source([1, _num_tx, tb_size])
#                         c = tb_encoder(b)
#                         x_map = _mapper(c)
#                         x_layer = _layer_mapper(x_map)
#                         x = _resource_grid_mapper(x_layer)

#                         y, h = channel_i([x, No])

#                         timestamp = datetime.now().strftime("%Y%m%d%H%M%S%f")
#                         save_pickle(b, f'{pickle_dir}/{timestamp}.b.pkl')
#                         save_pickle(c, f'{pickle_dir}/{timestamp}.c.pkl')
#                         save_pickle(y, f'{pickle_dir}/{timestamp}.y.pkl')

#                         pusch_records.append(PuschRecord(
#                                             nSFN=(sample_num // num_pusch_slots_in_frame) % 1023,
#                                             nSlot=slot_num,
#                                             nPDU=1,
#                                             nGroup=1,
#                                             nUlsch=1,
#                                             nUlcch=0,
#                                             nRachPresent=0,
#                                             nRNTI=_n_rnti,
#                                             nUEId=0,
#                                             nBWPSize=273,
#                                             nBWPStart=0,
#                                             nSubcSpacing=_mu,
#                                             nCpType=0,
#                                             nULType=0,
#                                             nMcsTable=0,
#                                             nMCS=mcs,
#                                             nTransPrecode=0,
#                                             nTransmissionScheme=0,
#                                             nNrOfLayers=_num_layers,
#                                             nPortIndex=[0],
#                                             nNid=_n_id,
#                                             nSCID=_n_scid,
#                                             nNIDnSCID=_n_id_n_scid,
#                                             nNrOfAntennaPorts=_num_rx_ant,
#                                             nVRBtoPRB=0,
#                                             nPMI=0,
#                                             nStartSymbolIndex=0,
#                                             nNrOfSymbols=14,
#                                             nResourceAllocType=1,
#                                             nRBStart=0,
#                                             nRBSize=273,
#                                             nTBSize=(tb_size/8).__ceil__(),
#                                             nRV=0,
#                                             nHARQID=sample_num % 16,
#                                             nNDI=1,
#                                             nMappingType=0,
#                                             nDMRSConfigType=0,
#                                             nNrOfCDMs=_num_cdm_groups_without_data,
#                                             nNrOfDMRSSymbols=_dmrs_length,
#                                             nDMRSAddPos=_dmrs_additional_position,
#                                             nPTRSPresent=0,
#                                             nAck=0,
#                                             nAlphaScaling=0,
#                                             nBetaOffsetACKIndex=0,
#                                             nCsiPart1=0,
#                                             nBetaOffsetCsiPart1Index=0,
#                                             nCsiPart2=0,
#                                             nBetaOffsetCsiPart2Index=0,
#                                             nTpPi2BPSK=0,
#                                             nTPPuschID=0,
#                                             nRxRUIdx=np.arange(0, _num_rx_ant),
#                                             nUE=1,
#                                             nPduIdx=[0],
#                                             Channel_model=f"CDL-{cdl_model}",
#                                             Speed=speed,
#                                             Delay_spread=delay_spread,
#                                             Esno_db=esno_db,
#                                             Data_filename=timestamp,
#                                             Data_dirname=name
#                                         )
#                                     )
#                         pbar.update(1)  # Increment progress

#                     df = pd.DataFrame.from_records(pusch_records, columns=PuschRecord._fields)
#                     df.to_parquet(f'{parquet_dir}/{name}.parquet', engine="pyarrow")
   

In [28]:
# import h5py
# import numpy as np
# import time
# import os

# # Generate sample dataset (simulating your b, c, y data)
# size_per_sample = 100 * 1024 * 1024  # ~100MB per dataset
# num_samples = 10  # Adjust for larger tests
# b = np.random.randint(0, 2, size=(num_samples, 1000000), dtype=np.uint8)
# c = np.random.randn(num_samples, 1000000).astype(np.float32)
# y = np.random.randn(num_samples, 1000000).astype(np.complex64)

# # Compression settings to test
# compression_methods = [
#     ("None", None, None),
#     ("Gzip 5", "gzip", 5),
#     ("LZF", "lzf", None)
# ]

# # Function to save data with different compression
# def save_hdf5(file_path, compression, level):
#     with h5py.File(file_path, "w") as f:
#         grp = f.create_group("pusch_data")
#         grp.create_dataset("b", data=b, compression=compression, compression_opts=level)
#         grp.create_dataset("c", data=c, compression=compression, compression_opts=level)
#         grp.create_dataset("y", data=y, compression=compression, compression_opts=level)

# # Function to load data and measure time
# def load_hdf5(file_path):
#     with h5py.File(file_path, "r") as f:
#         start_time = time.time()
#         _ = f["pusch_data"]["b"][:]
#         _ = f["pusch_data"]["c"][:]
#         _ = f["pusch_data"]["y"][:]
#         return time.time() - start_time

# # Run benchmark
# results = []
# for name, comp, level in compression_methods:
#     file_path = f"test_{name.replace(' ', '_')}.h5"

#     # Measure save time
#     start_time = time.time()
#     save_hdf5(file_path, comp, level)
#     save_time = time.time() - start_time

#     # Measure file size
#     file_size = os.path.getsize(file_path) / (1024**2)  # MB

#     # Measure load time
#     load_time = load_hdf5(file_path)

#     results.append((name, file_size, save_time, load_time))

#     print(f"{name}: Size = {file_size:.2f}MB | Save Time = {save_time:.2f}s | Load Time = {load_time:.2f}s")

# # Print final comparison
# print("\nFinal Results:")
# print("--------------------------------------------------")
# print("{:<10} | {:<10} | {:<10} | {:<10}".format("Method", "Size (MB)", "Save (s)", "Load (s)"))
# print("--------------------------------------------------")
# for name, file_size, save_time, load_time in results:
#     print(f"{name:<10} | {file_size:<10.2f} | {save_time:<10.2f} | {load_time:<10.2f}")


In [29]:
_binary_source = BinarySource(dtype=tf.float32)

_mapper = Mapper("qam", _num_bits_per_symbol, dtype=tf.complex64)

_layer_mapper = LayerMapper(num_layers=_num_layers, dtype=tf.complex64)

_pilot_pattern = PUSCHPilotPattern([_pusch_config_0],
                                        dtype=tf.complex64)
_mu = 1
_num_ofdm_symbols = 14
_fft_size = 4096
_cyclic_prefix_length = 288
_subcarrier_spacing = 30e3
_num_guard_subcarriers = (410+_n_start_bwp*12, _fft_size-_n_size_bwp*12-410)
_num_slots_per_frame = 20

In [30]:
# Define the resource grid.
_resource_grid = ResourceGrid(
    num_ofdm_symbols=_num_ofdm_symbols,
    fft_size=_fft_size,
    subcarrier_spacing=_subcarrier_spacing,
    num_tx=1,
    num_streams_per_tx=1,
    cyclic_prefix_length=_cyclic_prefix_length,
    num_guard_carriers=_num_guard_subcarriers,
    dc_null=False,
    pilot_pattern=_pilot_pattern,
    dtype=tf.complex64
)
_resource_grid_mapper = ResourceGridMapper(_resource_grid, dtype=tf.complex64)


In [31]:
tb_encoder = TBEncoder(
                    target_tb_size=_tb_size,
                    num_coded_bits=_num_coded_bits,
                    target_coderate=_target_coderate,
                    num_bits_per_symbol=_num_bits_per_symbol,
                    num_layers=_num_layers,
                    n_rnti=_n_rnti,
                    n_id=_n_id,
                    channel_type="PUSCH",
                    codeword_index=0,
                    use_scrambler=True,
                    verbose=False,
                    output_dtype=tf.float32)

In [32]:
# b = _binary_source([1, _num_tx, _tb_size])
b = tf.constant(generate_prng_seq(_tb_size, 2004*10+4)[None,None], dtype=tf.float32)
c = tb_encoder(b)
x_map = _mapper(c)
x_layer = _layer_mapper(x_map)
x = _resource_grid_mapper(x_layer)

In [33]:
np.sum(x[...,_num_guard_subcarriers[0]:-_num_guard_subcarriers[1]])

(-5.899493+23.798988j)

In [39]:
_channel_estimator = PUSCHLSChannelEstimator(
                _resource_grid,
                _dmrs_length,
                _dmrs_additional_position,
                _num_cdm_groups_without_data,
                interpolation_type='lin',
                dtype=tf.complex64)

_rx_tx_association = np.ones([_num_rx, _num_tx], bool)
_stream_management = StreamManagement(_rx_tx_association, _num_layers)
_mimo_detector = LinearDetector("lmmse", "bit", "maxlog", _resource_grid,
                                _stream_management, "qam", _num_bits_per_symbol, dtype=tf.complex64)



def generate_data(name,
        data_dir,
        mcss,
        channel_models,
        speeds,
        delay_spreads,
        esno_dbs,
        samples_per_case=1,
        save_dataset=True):

    def check_params():
        set(channel_models).issubset(['CDL-A', 'CDL-B', 'CDL-C', 'CDL-D', 'CDL-E', 'Umi-low', 'Umi-high', 'Uma-low', 'Uma-high'])

    check_params()

    pusch_records=[]
    parquet_dir = f'{data_dir}/parquet'
    if save_dataset: os.makedirs(parquet_dir, exist_ok=True)
    pickle_dir = f'{data_dir}/pickle/{name}'
    if save_dataset: os.makedirs(pickle_dir, exist_ok=True)

    total_iterations = len(mcss) * len(channel_models) * len(speeds) * len(delay_spreads) * len(esno_dbs) * samples_per_case
    with tqdm(total=total_iterations, desc="Generating Data") as pbar:
        for mcs in mcss:
            tb_size = mcs_params['tb_size'][mcs]
            num_coded_bits = mcs_params['num_coded_bits'][mcs]
            target_coderate = mcs_params['target_coderate'][mcs]
            num_bits_per_symbol = mcs_params['num_bits_per_symbol'][mcs]
            tb_encoder = TBEncoder(
                        target_tb_size=tb_size,
                        num_coded_bits=num_coded_bits,
                        target_coderate=target_coderate,
                        num_bits_per_symbol=num_bits_per_symbol,
                        num_layers=_num_layers,
                        n_rnti=_n_rnti,
                        n_id=_n_id,
                        channel_type="PUSCH",
                        codeword_index=0,
                        use_scrambler=True,
                        verbose=False,
                        output_dtype=tf.float32)

            for channel_model, speed, delay_spread in itertools.product(channel_models, speeds, delay_spreads):
                cm = channel_model.split('-')
                
                channel = cm[-2]
                model = cm[-1]

                if 'CDL' == channel:
                    channel_model_i = CDL(model = model,
                                          delay_spread = delay_spread,
                                          carrier_frequency = _carrier_frequency,
                                          ut_array = _ue_antenna,
                                          bs_array = _gnb_array,
                                          direction = _link_direction,
                                          min_speed = speed)

                else:
                  if 'Umi' == channel:
                      channel_model_i = UMi(carrier_frequency = _carrier_frequency,
                                          o2i_model = model,
                                          ut_array = _ue_antenna,
                                          bs_array = _gnb_array,
                                          direction = _link_direction,
                                          enable_pathloss = False)
                  elif 'Uma' == channel:
                      channel_model_i = UMa(carrier_frequency = _carrier_frequency,
                                            o2i_model = model,
                                            ut_array = _ue_antenna,
                                            bs_array = _gnb_array,
                                            direction = _link_direction,
                                            enable_pathloss = False)
                normalize = True if cm[0] == 'Normalized' else False
                channel_i = OFDMChannel(channel_model_i, _resource_grid, normalize_channel=normalize, return_channel=True)
                for esno_db in esno_dbs:
                    No = pow(10., -esno_db / 10.)
                    if channel in ['Umi', 'Uma']:
                        channel_i._cir_sampler.set_topology(*gen_topology(1,1,channel.lower(),min_ut_velocity=speed, max_ut_velocity=speed))


                    for sample_num in range(samples_per_case):
                        status_str = f"(MCS {mcs} | {channel_model} | {speed} m/s | {delay_spread} s) | {esno_db} dB | Sample {sample_num+1}/{samples_per_case}"
                        pbar.set_description(status_str)

                        slot_num = pusch_slots_in_frame[sample_num % num_pusch_slots_in_frame]
                        _resource_grid_mapper._resource_grid.pilot_pattern.pilots = slot_params['pilots'][sample_num % num_pusch_slots_in_frame]

                        b = _binary_source([1, _num_tx, tb_size])
                        c = tb_encoder(b)
                        x_map = _mapper(c)
                        x_layer = _layer_mapper(x_map)
                        x = _resource_grid_mapper(x_layer)

                        y, h = channel_i([x, No])

                        timestamp = datetime.now().strftime("%Y%m%d%H%M%S%f")

                        if save_dataset:
                            save_pickle(b, f'{pickle_dir}/{timestamp}.b.pkl')
                            save_pickle(c, f'{pickle_dir}/{timestamp}.c.pkl')
                            save_pickle(y, f'{pickle_dir}/{timestamp}.y.pkl')

                        pusch_records.append(PuschRecord(
                                            nSFN=(sample_num // num_pusch_slots_in_frame) % 1023,
                                            nSlot=slot_num,
                                            nPDU=1,
                                            nGroup=1,
                                            nUlsch=1,
                                            nUlcch=0,
                                            nRachPresent=0,
                                            nRNTI=_n_rnti,
                                            nUEId=0,
                                            nBWPSize=273,
                                            nBWPStart=0,
                                            nSubcSpacing=_mu,
                                            nCpType=0,
                                            nULType=0,
                                            nMcsTable=0,
                                            nMCS=mcs,
                                            nTransPrecode=0,
                                            nTransmissionScheme=0,
                                            nNrOfLayers=_num_layers,
                                            nPortIndex=[0],
                                            nNid=_n_id,
                                            nSCID=_n_scid,
                                            nNIDnSCID=_n_id_n_scid,
                                            nNrOfAntennaPorts=_num_rx_ant,
                                            nVRBtoPRB=0,
                                            nPMI=0,
                                            nStartSymbolIndex=0,
                                            nNrOfSymbols=14,
                                            nResourceAllocType=1,
                                            nRBStart=_n_start_bwp,
                                            nRBSize=_n_size_bwp,
                                            nTBSize=(tb_size/8).__ceil__(),
                                            nRV=0,
                                            nHARQID=sample_num % 16,
                                            nNDI=1,
                                            nMappingType=0,
                                            nDMRSConfigType=0,
                                            nNrOfCDMs=_num_cdm_groups_without_data,
                                            nNrOfDMRSSymbols=_dmrs_length,
                                            nDMRSAddPos=_dmrs_additional_position,
                                            nPTRSPresent=0,
                                            nAck=0,
                                            nAlphaScaling=0,
                                            nBetaOffsetACKIndex=0,
                                            nCsiPart1=0,
                                            nBetaOffsetCsiPart1Index=0,
                                            nCsiPart2=0,
                                            nBetaOffsetCsiPart2Index=0,
                                            nTpPi2BPSK=0,
                                            nTPPuschID=0,
                                            nRxRUIdx=np.arange(0, _num_rx_ant),
                                            nUE=1,
                                            nPduIdx=[0],
                                            Channel_model=f"{channel_model}",
                                            Speed=speed,
                                            Delay_spread=delay_spread,
                                            Esno_db=esno_db,
                                            Data_filename=timestamp,
                                            Data_dirname=name
                                        )
                                    )
                        pbar.update(1)  # Increment progress

                    df = pd.DataFrame.from_records(pusch_records, columns=PuschRecord._fields)
                    if save_dataset: df.to_parquet(f'{parquet_dir}/{name}.parquet', engine="pyarrow")
    return df

In [40]:
# samples_per_case = 1

data_dir = '/workspaces/thanh/Pusch_data'
# os.makedirs(data_dir,exist_ok=True)
# os.makedirs(f'{data_dir}/pickle',exist_ok=True)
# os.makedirs(f'{data_dir}/parquet',exist_ok=True)

name = datetime.now().strftime("%Y%m%d%H%M%S%f")

name = '4RB_test'
generate_data(name=f'{name}',
              data_dir=data_dir,
              mcss=[3],
              channel_models=['Normalized-CDL-B'],
              speeds=[1],
              delay_spreads=[10e-9],
              esno_dbs=[i for i in np.arange(5,5.1,5)],
              samples_per_case=4,
              save_dataset=False
)

Generating Data:   0%|          | 0/4 [00:00<?, ?it/s]

,nSFN,nSlot,nPDU,nGroup,nUlsch,nUlcch,nRachPresent,nRNTI,nUEId,nBWPSize,...,nTPPuschID,nRxRUIdx,nUE,nPduIdx,Channel_model,Speed,Delay_spread,Esno_db,Data_filename,Data_dirname
0,0,4,1,1,1,0,0,2008,0,273,...,0,"[0, 1, 2, 3, 4, 5, 6, 7]",1,[0],Normalized-CDL-B,1,1.000000e-08,5.0,20250227144526286373,4RB_test
1,0,5,1,1,1,0,0,2008,0,273,...,0,"[0, 1, 2, 3, 4, 5, 6, 7]",1,[0],Normalized-CDL-B,1,1.000000e-08,5.0,20250227144526751008,4RB_test
2,0,14,1,1,1,0,0,2008,0,273,...,0,"[0, 1, 2, 3, 4, 5, 6, 7]",1,[0],Normalized-CDL-B,1,1.000000e-08,5.0,20250227144527162853,4RB_test
3,0,15,1,1,1,0,0,2008,0,273,...,0,"[0, 1, 2, 3, 4, 5, 6, 7]",1,[0],Normalized-CDL-B,1,1.000000e-08,5.0,20250227144527660148,4RB_test


In [ ]:
# drive_parquet_folder_id = "1p9e9tw9iFrXiaFFh_TP-cuaHwxwLYMYb"
# drive_pickle_folder_id = "1_kx1wUO8eBe7tltrYQhWWNFhS8dbfDFW"

# def create_drive_folder(folder_name, parent_folder_id=None):
#     """Creates a folder in Google Drive and returns the folder ID."""
#     folder_metadata = {
#         "name": folder_name,
#         "mimeType": "application/vnd.google-apps.folder",
#     }
#     if parent_folder_id:
#         folder_metadata["parents"] = [parent_folder_id]

#     folder = drive_service.files().create(body=folder_metadata, fields="id").execute()
#     print(f'Folder "{folder_name}" created with ID: {folder["id"]}')
#     return folder["id"]

# def upload_files_to_drive(data_dir, name, folder_ids):
#     files_to_upload = [f'{data_dir}/parquet/{name}.parquet']
#     for (root, _, file) in os.walk(f'{data_dir}/pickle/{name}'):
#         for f in file:
#             files_to_upload.append(os.path.join(root, f))

#     pickle_subfolder_id = create_drive_folder(name, folder_ids[1])
#     """Uploads multiple files sequentially but with optimizations."""
#     for file_path in tqdm(files_to_upload, desc="Uploading Files"):
#         file_name = file_path.split("/")[-1]
#         folder_id = folder_ids[0] if ".parquet" in file_name else pickle_subfolder_id

#         file_metadata = {"name": file_name, "parents": [folder_id]}
#         media = MediaFileUpload(file_path, resumable=True)  # Enables resumable uploads

#         try:
#             drive_service.files().create(body=file_metadata, media_body=media, fields="id").execute()
#         except Exception as e:
#             print(f"Error uploading {file_name}: {e}")

#     # if os.path.exists(data_dir): os.rename(data_dir, '../Dataset/'+datetime.now().strftime('%Y%m%d%H%M'))
# # Example Usage
# data_dir = '../Dataset/tmp'
# upload_filename = name
# upload_files_to_drive(data_dir, upload_filename, [drive_parquet_folder_id, drive_pickle_folder_id])


NameError: name 'dadada' is not defined

Hopefully you have enjoyed this tutorial on Sionna's 5G NR PUSCH module!

Please have a look at the [API documentation](https://nvlabs.github.io/sionna/api/sionna.html) of the various components or the other available [tutorials](https://nvlabs.github.io/sionna/tutorials.html) to learn more.